# <b> Montreal Neighborhood Analysis </b>
## by Jing Han Sun

<ul>
    <li>Build a dataframe of Montreal's boroughs and their coordinates</li>
    <li>Obtain some venue data from Foursquare API</li>
    <li>Analyse each neighborhood</li>
    <li>Cluster neighborhoods</li>
    <li>Examine clusters</li>
</ul>

Import Libraries

In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes ted the Foursquare API lab
import folium 

print('libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.5
  latest version: 4.9.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9880bd3_2         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         248 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0

In [2]:
!conda install -c conda-forge bs4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.5
  latest version: 4.9.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - bs4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.3       |     pyhb0f4dca_0          86 KB  conda-forge
    bs4-4.9.3                  |                0           4 KB  conda-forge
    soupsieve-2.0.1            |             py_1          30 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         120 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/noarch::beautifulsoup4-4.9.3-pyhb0f4dca_0
  bs4                conda-forge/noarch::bs4

In [3]:
from bs4 import BeautifulSoup

#import the table
url = requests.get('https://en.wikipedia.org/wiki/Boroughs_of_Montreal').text
soup = BeautifulSoup(url,'html.parser')

Make DataFrame

In [4]:
#create list for neighboorhood
borough = []
density = []

#locate html tags to filter table
#tr = rows, td = cells
for row in soup.find('table').find_all('tr'):
    cell = row.find_all('td')
    if(len(cell) > 0): 
        borough.append(cell[1].text.rstrip('\n')) #remove the enters at the end of each cell
        density.append(cell[4].text.rstrip('\n'))

In [5]:
#create dataframe

df = pd.DataFrame({"Borough": borough, "Density": density})
df.head()

,Borough,Density
0,Ahuntsic-Cartierville,"5,547.3"
1,Anjou,"3,123.8"
2,Côte-des-Neiges–Notre-Dame-de-Grâce,"7,781.3"
3,Lachine,"2,513.5"
4,LaSalle,"4,714.9"


In [6]:
#sort boroughs
df = df.sort_values(by=['Borough'])
df = df.reset_index(drop=True)
df.head(20)

,Borough,Density
0,Ahuntsic-Cartierville,"5,547.3"
1,Anjou,"3,123.8"
2,Côte-des-Neiges–Notre-Dame-de-Grâce,"7,781.3"
3,L'Île-Bizard–Sainte-Geneviève,780.2
4,LaSalle,"4,714.9"
5,Lachine,"2,513.5"
6,Le Plateau-Mont-Royal,"12,839.5"
7,Le Sud-Ouest,"4,977.8"
8,Mercier–Hochelaga-Maisonneuve,"5,355.3"
9,Montréal-Nord,"7,588.6"


In [7]:
#list of latitudes
latitude = [45.5447, 45.6160, 45.4912, 45.4949, 45.4306, 45.4413, 45.5232, 45.4661, 45.5730, \
            45.6072, 45.5143, 45.4688, 45.6731, 45.5517, 45.4986, 45.5875, 45.4548, 45.5088, \
            45.5553]
#list of longitudes
longitude = [-73.6681, -73.5694, -73.6327, -73.8908, -73.6348, -73.6886, -73.5870, -73.5939,  \
             -73.5368, -73.6315, -73.6090, -73.8756, -73.5187, -73.5836, -73.7498, -73.5970, \
             -73.5699, -73.5553, -73.6083]

In [8]:
#insert the latitude and longitude data into the dataframe
df['Latitude'] = latitude
df['Longitude'] = longitude

df.head()

,Borough,Density,Latitude,Longitude
0,Ahuntsic-Cartierville,"5,547.3",45.5447,-73.6681
1,Anjou,"3,123.8",45.6160,-73.5694
2,Côte-des-Neiges–Notre-Dame-de-Grâce,"7,781.3",45.4912,-73.6327
3,L'Île-Bizard–Sainte-Geneviève,780.2,45.4949,-73.8908
4,LaSalle,"4,714.9",45.4306,-73.6348


In [9]:
#import the average_family_income.csv as dataset
df_afi = pd.read_csv('average_family_income.csv', usecols = ['Borough', 'Average Income', 'Rank'])
df_afi.head()

,Borough,Average Income,Rank
0,Outremont,150697,1
1,L'Île-Bizard–Sainte-Geneviève,114936,2
2,Ville-Marie,96085,3
3,Verdun,94026,4
4,Pierrefonds-Roxboro,88032,5


In [10]:
#the 4 other files
df_er = pd.read_csv('employment_rate.csv', usecols = ['Borough', 'Employment Rate', 'Rank'])
df_hp = pd.read_csv('housing_prices.csv', usecols = ['Borough', 'Average Housing Price', 'Rank'])
df_ir = pd.read_csv('immigration.csv', usecols = ['Borough', 'Immigration Rate', 'Rank'])
df_mct = pd.read_csv('median_commute_time.csv', usecols = ['Borough', 'Median Commute Time', 'Rank'])


In [11]:
#drop the rank columns and sort the boroughs in alphabetical order
df_afi = df_afi.drop(columns = ['Rank'])
df_afi = df_afi.sort_values(by=['Borough'])
df_afi = df_afi.drop(columns = ['Borough'])
df_afi = df_afi.reset_index(drop=True)

In [12]:
df_er = df_er.drop(columns = ['Rank'])
df_er = df_er.sort_values(by=['Borough'])
df_er = df_er.drop(columns = ['Borough'])
df_er = df_er.reset_index(drop=True)

In [13]:
df_hp = df_hp.drop(columns = ['Rank'])
df_hp = df_hp.sort_values(by=['Borough'])
df_hp = df_hp.drop(columns = ['Borough'])
df_hp = df_hp.reset_index(drop=True)

In [14]:
df_ir = df_ir.drop(columns = ['Rank'])
df_ir = df_ir.sort_values(by=['Borough'])
df_ir = df_ir.drop(columns = ['Borough'])
df_ir = df_ir.reset_index(drop=True)

In [15]:
df_mct = df_mct.drop(columns = ['Rank'])
df_mct = df_mct.sort_values(by=['Borough'])
df_mct = df_mct.drop(columns = ['Borough'])
df_mct = df_mct.reset_index(drop=True)

In [16]:
#verify
df_mct.head()

,Median Commute Time
0,30.2
1,30.2
2,30.2
3,30.1
4,30.1


In [17]:
df_ir.head()

,Immigration Rate
0,38.9
1,27.9
2,47.7
3,22.4
4,34.7


In [18]:
#join all the dataframes into a single one
new_df = pd.concat([df, df_afi, df_er, df_hp, df_ir, df_mct], axis=1)
new_df.head()

,Borough,Density,Latitude,Longitude,Average Income,Employment Rate,Average Housing Price,Immigration Rate,Median Commute Time
0,Ahuntsic-Cartierville,"5,547.3",45.5447,-73.6681,82437,54.8,397121,38.9,30.2
1,Anjou,"3,123.8",45.6160,-73.5694,74367,50.3,319918,27.9,30.2
2,Côte-des-Neiges–Notre-Dame-de-Grâce,"7,781.3",45.4912,-73.6327,75169,56.1,459753,47.7,30.2
3,L'Île-Bizard–Sainte-Geneviève,780.2,45.4949,-73.8908,114936,67.7,378992,22.4,30.1
4,LaSalle,"4,714.9",45.4306,-73.6348,66841,53.5,376936,34.7,30.1


In [19]:
new_df.shape

(19, 9)

In [20]:
# save the dataframe as CSV file
new_df.to_csv("mtl_df.csv", index=False)

Use Foursquare API and extract some venues

In [21]:
#get latitude and logitude of Montreal
address = 'Montreal, Canada'

geolocator = Nominatim(user_agent='montreal-explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Montreal are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Montreal are 45.4972159, -73.6103642.


In [22]:
# create map of montreal using latitude and longitude values
map_mtl = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(df['Latitude'], df['Longitude'], df['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mtl)  
    
map_mtl

In [23]:
#save the map as HTML file
map_mtl.save('map_mtl.html')

In [24]:
#use foursquare api to explore neighborhoods

# define Foursquare Credentials and Version
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CLIENT_SECRET:

In [25]:
#get the top 100 venues in Montreal within a 1500 m radius

radius = 1500
LIMIT = 100

venues_list = []

for lat, long, borough, density, ai, er, hp, ir, mct in zip(new_df['Latitude'], new_df['Longitude'], new_df['Borough'], \
                                                            new_df['Density'], new_df['Average Income'], new_df['Employment Rate'], \
                                                            new_df['Average Housing Price'], new_df['Immigration Rate'], \
                                                            new_df['Median Commute Time']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    venues = requests.get(url).json()['response']['groups'][0]['items']

    for venue in venues:
        venues_list.append(( 
            borough,
            lat, 
            long,
            density, ai, er, hp, ir, mct,
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [26]:
#convert it into a dataframe

venues_df = pd.DataFrame(venues_list, columns = ['Borough', 'BoroughLatitude', 'BoroughLongitude', 'BoroughDensity', \
                                                 'AverageIncome', 'EmploymentRate', 'HousingPrice', 'Immigration', \
                                                 'CommuteTime', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'Category'])
venues_df.head()

,Borough,BoroughLatitude,BoroughLongitude,BoroughDensity,AverageIncome,EmploymentRate,HousingPrice,Immigration,CommuteTime,VenueName,VenueLatitude,VenueLongitude,Category
0,Ahuntsic-Cartierville,45.5447,-73.6681,"5,547.3",82437,54.8,397121,38.9,30.2,Les Cavistes Fleury,45.545784,-73.666446,Wine Bar
1,Ahuntsic-Cartierville,45.5447,-73.6681,"5,547.3",82437,54.8,397121,38.9,30.2,Le Brûloir,45.543601,-73.667883,Café
2,Ahuntsic-Cartierville,45.5447,-73.6681,"5,547.3",82437,54.8,397121,38.9,30.2,132 Bar Vintage,45.546571,-73.665895,Lounge
3,Ahuntsic-Cartierville,45.5447,-73.6681,"5,547.3",82437,54.8,397121,38.9,30.2,La Bête à Pain,45.546803,-73.665857,Bakery
4,Ahuntsic-Cartierville,45.5447,-73.6681,"5,547.3",82437,54.8,397121,38.9,30.2,La Consigne,45.546059,-73.666283,Beer Store


In [27]:
venues_df.shape

(1078, 13)

Analyse each neighborhood

In [28]:
#check number of venues per borough
venues_df.groupby(['Borough']).count()

,BoroughLatitude,BoroughLongitude,BoroughDensity,AverageIncome,EmploymentRate,HousingPrice,Immigration,CommuteTime,VenueName,VenueLatitude,VenueLongitude,Category
Borough,,,,,,,,,,,,
Ahuntsic-Cartierville,75,75,75,75,75,75,75,75,75,75,75,75
Anjou,22,22,22,22,22,22,22,22,22,22,22,22
Côte-des-Neiges–Notre-Dame-de-Grâce,100,100,100,100,100,100,100,100,100,100,100,100
L'Île-Bizard–Sainte-Geneviève,12,12,12,12,12,12,12,12,12,12,12,12
LaSalle,40,40,40,40,40,40,40,40,40,40,40,40
Lachine,41,41,41,41,41,41,41,41,41,41,41,41
Le Plateau-Mont-Royal,100,100,100,100,100,100,100,100,100,100,100,100
Le Sud-Ouest,72,72,72,72,72,72,72,72,72,72,72,72
Mercier–Hochelaga-Maisonneuve,51,51,51,51,51,51,51,51,51,51,51,51


In [29]:
#check the number of unique categories
print('There are {} uniques categories.'.format(len(venues_df['Category'].unique())))

There are 194 uniques categories.


In [30]:
#analyze each neighborhood

# one hot encoding
mtl_onehot = pd.get_dummies(venues_df['Category'], prefix="", prefix_sep="")

# add postal code, borough, neighborhood column back to dataframe
mtl_onehot['Borough'] = venues_df['Borough']

# move postal code, borough, neighborhood column to the first column
fixed_columns = [mtl_onehot.columns[-1]] + list(mtl_onehot.columns[:-1])
mtl_onehot = mtl_onehot[fixed_columns]

mtl_onehot.head()

,Borough,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beer Bar,Beer Store,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Candy Store,Caribbean Restaurant,Casino,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Drugstore,English Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hockey Arena,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lighthouse,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motorsports Shop,Movie Theater,Multiplex,Music Venue,New American Restaurant,Nightclub,Outdoor Sculpture,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Real Estate Office,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Soccer Field,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Women's Store,Yoga Studio
0,Ahuntsic-Cartierville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,Ahuntsic-Cartierville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ahuntsic-Cartierville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [31]:
#group rows by area and by taking the mean of the frequency of occurrence of each category
mtl_grouped = mtl_onehot.groupby(['Borough']).mean().reset_index()
print(mtl_grouped.shape)
mtl_grouped.head()

(19, 195)


,Borough,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beer Bar,Beer Store,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Candy Store,Caribbean Restaurant,Casino,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Drugstore,English Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hockey Arena,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lighthouse,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motorsports Shop,Movie Theater,Multiplex,Music Venue,New American Restaurant,Nightclub,Outdoor Sculpture,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Real Estate Office,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Soccer Field,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Women's Store,Yoga Studio
0,Ahuntsic-Cartierville,0.013333,0.0,0.00,0.0,0.013333,0.013333,0.000000,0.0,0.0,0.000000,0.0,0.013333,0.013333,0.00,0.013333,0.0,0.0,0.013333,0.0,0.0,0.000,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.0,0.0,0.00,0.040,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.013333,0.013333,0.0,0.0,0.0,0.053333,0.0,0.013333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.013333,0.013333,0.013333,0.0,0.013333,0.013333,0.00,0.000000,0.00,0.0,0.0,0.0,0.013333,0.040000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.013333,0.0,0.0,0.0,0.000000,0.0,0.013333,0.040000,0.000000,0.000,0.0,0.0,0.0,0.013333,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.013333,0.000,0.0,0.0,0.0,0.000000,0.0,0.04,0.0,0.0,0.0,0.013333,0.013333,0.0,0.0,0.026667,0.013333,0.0,0.0,0.0,0.0,0.026667,0.000000,0.00,0.00,0.0,0.013333,0.0,0.0,0.0,0.013333,0.026667,0.0,0.0,0.0,0.0,0.013333,0.066667,0.0,0.040000,0.0,0.0,0.0,0.0,0.013333,0.00,0.0,0.00,0.00,0.0,0.0,0.000000,0.013333,0.00,0.040000,0.00,0.0,0.00,0.0,0.040000,0.0,0.0,0.013333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013333,0.0,0.00,0.0,0.000,0.013333,0.0,0.0,0.0,0.013333,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.013333,0.013333,0.013333,0.013333,0.0
1,Anjou,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.045455,0.0,0.0000

In [32]:
#function that sorts the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
#display top 10 venues of each borough
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = mtl_grouped['Borough']

for ind in np.arange(mtl_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mtl_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ahuntsic-Cartierville,Pharmacy,Clothing Store,Pizza Place,Fast Food Restaurant,Café,Breakfast Spot,Sandwich Place,Italian Restaurant,Restaurant,Grocery Store
1,Anjou,Coffee Shop,Gas Station,Pizza Place,Golf Course,Fast Food Restaurant,Sandwich Place,Drugstore,Food & Drink Shop,Restaurant,Furniture / Home Store
2,Côte-des-Neiges–Notre-Dame-de-Grâce,Vietnamese Restaurant,Coffee Shop,Pharmacy,Chinese Restaurant,Bakery,Park,Bank,Fast Food Restaurant,Department Store,Clothing Store
3,L'Île-Bizard–Sainte-Geneviève,Golf Course,Hockey Arena,Grocery Store,Pizza Place,Pharmacy,Park,Sandwich Place,French Restaurant,Gas Station,Convenience Store
4,LaSalle,Fast Food Restaurant,Pizza Place,Grocery Store,Bank,Italian Restaurant,Coffee Shop,Sandwich Place,Breakfast Spot,Pharmacy,Chinese Restaurant


Cluster neighborhoods

In [34]:
#Run k-means to cluster the neighborhood into 4 clusters

# set number of clusters
kclusters = 4

#add sociodemographic data into clustering
mtl_grouped_clustering = new_df[['Borough', 'Average Income', 'Employment Rate', \
                                'Average Housing Price', 'Immigration Rate', \
                                'Median Commute Time']].join(mtl_grouped.set_index('Borough'), on='Borough')

mtl_grouped_clustering = mtl_grouped_clustering.drop(['Borough'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(mtl_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 0, 1, 3, 3, 0, 3, 0, 0, 0], dtype=int32)

In [35]:
# dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mtl_merged = new_df

mtl_merged = mtl_merged.join(neighborhoods_venues_sorted.set_index('Borough'), on='Borough')

mtl_merged.head()

,Borough,Density,Latitude,Longitude,Average Income,Employment Rate,Average Housing Price,Immigration Rate,Median Commute Time,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ahuntsic-Cartierville,"5,547.3",45.5447,-73.6681,82437,54.8,397121,38.9,30.2,3,Pharmacy,Clothing Store,Pizza Place,Fast Food Restaurant,Café,Breakfast Spot,Sandwich Place,Italian Restaurant,Restaurant,Grocery Store
1,Anjou,"3,123.8",45.6160,-73.5694,74367,50.3,319918,27.9,30.2,0,Coffee Shop,Gas Station,Pizza Place,Golf Course,Fast Food Restaurant,Sandwich Place,Drugstore,Food & Drink Shop,Restaurant,Furniture / Home Store
2,Côte-des-Neiges–Notre-Dame-de-Grâce,"7,781.3",45.4912,-73.6327,75169,56.1,459753,47.7,30.2,1,Vietnamese Restaurant,Coffee Shop,Pharmacy,Chinese Restaurant,Bakery,Park,Bank,Fast Food Restaurant,Department Store,Clothing Store
3,L'Île-Bizard–Sainte-Geneviève,780.2,45.4949,-73.8908,114936,67.7,378992,22.4,30.1,3,Golf Course,Hockey Arena,Grocery Store,Pizza Place,Pharmacy,Park,Sandwich Place,French Restaurant,Gas Station,Convenience Store
4,LaSalle,"4,714.9",45.4306,-73.6348,66841,53.5,376936,34.7,30.1,3,Fast Food Restaurant,Pizza Place,Grocery Store,Bank,Italian Restaurant,Coffee Shop,Sandwich Place,Breakfast Spot,Pharmacy,Chinese Restaurant


In [36]:
#visualize clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = ['red', 'green', 'blue', 'purple']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mtl_merged['Latitude'], mtl_merged['Longitude'], mtl_merged['Borough'], mtl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-4],
        fill=True,
        fill_color=rainbow[cluster-4],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

Examine Clusters

In [37]:
#cluster 1
mtl_merged.loc[mtl_merged['Cluster Labels'] == 0]

,Borough,Density,Latitude,Longitude,Average Income,Employment Rate,Average Housing Price,Immigration Rate,Median Commute Time,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Anjou,"3,123.8",45.6160,-73.5694,74367,50.3,319918,27.9,30.2,0,Coffee Shop,Gas Station,Pizza Place,Golf Course,Fast Food Restaurant,Sandwich Place,Drugstore,Food & Drink Shop,Restaurant,Furniture / Home Store
5,Lachine,"2,513.5",45.4413,-73.6886,75999,58.5,338447,21.3,20.7,0,Pharmacy,Park,Italian Restaurant,Restaurant,Grocery Store,Café,Ice Cream Shop,Pier,Farmers Market,Brewery
7,Le Sud-Ouest,"4,977.8",45.4661,-73.5939,65166,56.1,321431,23.9,25.8,0,Bakery,Park,Breakfast Spot,Pizza Place,Café,Restaurant,Ice Cream Shop,French Restaurant,Brewery,Coffee Shop
8,Mercier–Hochelaga-Maisonneuve,"5,355.3",45.5730,-73.5368,67259,57.8,319325,18.8,30.2,0,Coffee Shop,Italian Restaurant,Restaurant,Thai Restaurant,Greek Restaurant,Park,Fast Food Restaurant,Breakfast Spot,Pharmacy,Paintball Field
9,Montréal-Nord,"7,588.6",45.6072,-73.6315,53525,47.3,321517,37.6,30.3,0,Pharmacy,Breakfast Spot,Park,Italian Restaurant,Supermarket,Fast Food Restaurant,Video Game Store,Coffee Shop,Grocery Store,Hardware Store
11,Pierrefonds-Roxboro,"2,557.1",45.4688,-73.8756,88032,61.6,318595,38.2,25.9,0,Coffee Shop,Pizza Place,Pharmacy,Convenience Store,Grocery Store,Ice Cream Shop,Soccer Field,Park,Liquor Store,Sandwich Place
12,Rivière-des-Prairies–Pointe-aux-Trembles,"2,523.2",45.6731,-73.5187,76941,58.1,273699,22.7,30.3,0,Theater,Convenience Store,Business Service,Supermarket,Sushi Restaurant,Yoga Studio,Drugstore,Filipino Restaurant,Fast Food Restaurant,Farmers Market
18,Villeray–Saint-Michel–Parc-Extension,"8,718.3",45.5553,-73.6083,55108,53.0,348980,43.9,30.3,0,Vietnamese Restaurant,Pharmacy,Sushi Restaurant,Italian Restaurant,Bakery,Coffee Shop,Café,Portuguese Restaurant,Grocery Store,Sandwich Place


In [38]:
#cluster 2
mtl_merged.loc[mtl_merged['Cluster Labels'] == 1]

,Borough,Density,Latitude,Longitude,Average Income,Employment Rate,Average Housing Price,Immigration Rate,Median Commute Time,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Côte-des-Neiges–Notre-Dame-de-Grâce,"7,781.3",45.4912,-73.6327,75169,56.1,459753,47.7,30.2,1,Vietnamese Restaurant,Coffee Shop,Pharmacy,Chinese Restaurant,Bakery,Park,Bank,Fast Food Restaurant,Department Store,Clothing Store
15,Saint-Léonard,"5,800.4",45.5875,-73.5970,64411,50.0,480222,46.3,30.3,1,Pharmacy,Gas Station,Grocery Store,Italian Restaurant,Fast Food Restaurant,Restaurant,Drugstore,Sandwich Place,Bank,Deli / Bodega
17,Ville-Marie,"5,404.2",45.5088,-73.5553,96085,57.4,438492,32.5,20.7,1,Café,Restaurant,Plaza,Hotel,Coffee Shop,French Restaurant,Spa,Italian Restaurant,Tea Room,Sandwich Place


In [39]:
#cluster 3
mtl_merged.loc[mtl_merged['Cluster Labels'] == 2]

,Borough,Density,Latitude,Longitude,Average Income,Employment Rate,Average Housing Price,Immigration Rate,Median Commute Time,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Outremont,"6,142.0",45.5143,-73.609,150697,63.3,678912,23.3,20.7,2,Café,Bakery,Supermarket,Restaurant,French Restaurant,Park,Breakfast Spot,Ice Cream Shop,Bar,Bagel Shop


In [40]:
#cluster 4
mtl_merged.loc[mtl_merged['Cluster Labels'] == 3]

,Borough,Density,Latitude,Longitude,Average Income,Employment Rate,Average Housing Price,Immigration Rate,Median Commute Time,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ahuntsic-Cartierville,"5,547.3",45.5447,-73.6681,82437,54.8,397121,38.9,30.2,3,Pharmacy,Clothing Store,Pizza Place,Fast Food Restaurant,Café,Breakfast Spot,Sandwich Place,Italian Restaurant,Restaurant,Grocery Store
3,L'Île-Bizard–Sainte-Geneviève,780.2,45.4949,-73.8908,114936,67.7,378992,22.4,30.1,3,Golf Course,Hockey Arena,Grocery Store,Pizza Place,Pharmacy,Park,Sandwich Place,French Restaurant,Gas Station,Convenience Store
4,LaSalle,"4,714.9",45.4306,-73.6348,66841,53.5,376936,34.7,30.1,3,Fast Food Restaurant,Pizza Place,Grocery Store,Bank,Italian Restaurant,Coffee Shop,Sandwich Place,Breakfast Spot,Pharmacy,Chinese Restaurant
6,Le Plateau-Mont-Royal,"12,839.5",45.5232,-73.5870,83803,67.7,402662,24.3,25.3,3,Café,Bakery,Coffee Shop,Vegetarian / Vegan Restaurant,French Restaurant,Portuguese Restaurant,Park,Yoga Studio,Dessert Shop,Breakfast Spot
13,Rosemont–La Petite-Patrie,"8,779.2",45.5517,-73.5836,72119,61.8,363599,21.8,30.2,3,Pharmacy,Bakery,Café,Breakfast Spot,Grocery Store,Bar,Italian Restaurant,Sushi Restaurant,Park,Pizza Place
14,Saint-Laurent,"2,309.1",45.4986,-73.7498,80672,54.7,400288,52.4,25.7,3,Park,Coffee Shop,Arts & Crafts Store,Gas Station,Restaurant,Yoga Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
16,Verdun,"7,137.0",45.4548,-73.5699,94026,59.3,412770,22.5,25.8,3,Grocery Store,Park,Pizza Place,Café,Restaurant,Bakery,Indian Restaurant,Pharmacy,Fish & Chips Shop,Italian Restaurant


<ul>
    <li>Cluster 1: Average residential area with a reasonable range of activities nearby. </li>
    <li>Cluster 2: More populated areas with many nearby activities. </li>
Cluster 3: Neighborhoods with a higher cost of living in the center of the island.
Cluster 4: Areas with medium-to-high cost of living a bit farther from the center.
    </ul>